# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [8]:
import pandas as pd
import numpy as np
DATA_FOLDER = './data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Library

In [9]:
# To list file and folder dynamically
from os import listdir

## Functions

In [10]:
""" 
    Function to merge all data into one single dataframe
    @params: (string) directory, path to ebola data
    @params: (list of string) list_countries, available countries that we want to crawl data for
    return: (panda dataframe) data containing all ebola information
"""
def crawl_data(directory, list_countries):
    data = []

    for country in list_countries:
        # list all available data per country
        country_data_files = listdir(directory + "/" + country + "_data")
        country_data = []
        
        # for each csv file, read and store it to single list
        for country_data_file in country_data_files:
            # get the absoulte path of each csv file
            csv_file = (directory + "/" + country + "_data/" + country_data_file)
            
            # read data, use "Date" (column 0) as index, parse dates format, and make "-" character as NaN
            data_per_country_date = pd.read_csv(csv_file, index_col=[0], parse_dates=[0], na_values=['-'])
            
            # append to list
            country_data.append(data_per_country_date)
        
        # merge one country data as one single dataframe
        country_data = pd.concat(country_data)
        # append the country's dataframe into list
        data.append(country_data)

    # Form a single DataFrame for all countries, use countries' name as keys    
    data = pd.concat(data,keys=list_countries)
    # Name the index
    data.index.names = ["Country","Date"]
    # return the formed dataframe
    return data

""" 
    Function to get the daily average per month of new cases and new deaths for a country
    @params: (panda dataframe) data, dataframe containing all sources of information
    @params: (string) country, the name of country
    @params: (string) col_descriptor, the name of column used to describe records
    @params: (list of string) keywords, the descriptions that we want to account for
    @params: (string) field, the column that we "assume" contain the sum of a record
    return: (panda series) a series containing data of daily average per month for given keywords
"""
def extract_data(data, country, col_descriptor, keywords, field):
    # find the index of given country, filter based on keywords found in col_descriptor
    index = data.loc[country][col_descriptor].isin(keywords)
    # reduce the big dataframe into df, fill NaN with zeros
    df = (data.loc[country][index].fillna(value=0.0))
    
    # find the information for month and year of country's data
    date = df.index.month
    year = df.index.year
    
    # create two dataframe: 
    # df1 is used to calculate based on sum of all columns per record (per row)
    # df2 is used to calculate based on "assumed" column that contains sum per category
    df1 = df
    df2 = df
    
    ######################################################
    # throw the column that "assumed" contains total values per record (per row)
    df1 = df1.drop([field],axis=1)
    
    # throw the column of descriptor, since we have used it to form df
    df1 = df1.drop([col_descriptor],axis=1)
    
    # change the entries into float
    df1 = df1.apply(pd.to_numeric)
    
    # sum the data per record (per row)
    df1 = df1.sum(axis=1)
    
    # group the data based on month,year and sum it based on the grouping
    df1 = (pd.Series.groupby(df1,by=[date,year])).sum()
    
    # give names to series index
    df1.index.names = ["Month","Year"]
    # finish forming series df1, contains sum of all columns per record (per row)
    ######################################################
    
    ######################################################
    # change the datatype of column that "assumed" contains total values per record (per row)
    df2[field] = df2[field].astype(float)
    
    # group the data based on month,year and sum it "assumed" column
    df2 = (pd.DataFrame.groupby(df2,by=[date,year]))[field].sum()
    
    # give names to series index
    df2.index.names = ["Month","Year"]
    # finish forming series df2, contains "assumed" sum of data per record (per row)
    ######################################################
    
    """
        Consider both result by comparing series df1 and df2
        We assume that the largest entry per index represent the real sum of data per records
    """
    
    # index1 contains all true indexs in which df1 > df2
    index1 = (df1 >= df2)
    
    # index2 contains all true indexs in which df2 > df1
    index2 = (~index1)
    
    # form the final result by adding both df and dividing by 30 (average per day, 1 month=30 days)
    result_series = (df1[index1].add(df2[index2], fill_value=0))/30
    
    return result_series

## Vars

In [11]:
# Assign Static Values
EBOLA_DATA_DIR = DATA_FOLDER + "/ebola"

# list countries dynamically
countries = [c[:-5] for c in listdir(EBOLA_DATA_DIR) if c != "LICENSE"]

# save all data into single panda dataframe var
ebola_data = crawl_data(EBOLA_DATA_DIR,countries)

## Results

In [12]:
### For guinea ###
# Keywords for New cases: New cases of suspects, New cases of probables, New cases of confirmed
# Keywords for New deaths: New deaths registered
# Column "Description" gives detail of each record (row)
# Column "Totals" is assumed to contain the sum of data per description
guinea_new_cases_label = ["New cases of suspects","New cases of probables", "New cases of confirmed"]
guinea_new_deaths_label = [ "New deaths registered" ]

guinea_new_cases = extract_data(ebola_data, "guinea", "Description", guinea_new_cases_label, "Totals")
guinea_new_deaths = extract_data(ebola_data, "guinea", "Description", guinea_new_deaths_label, "Totals")

print(guinea_new_cases)
print(guinea_new_deaths)

Month  Year
8      2014     4.300000
9      2014    10.466667
10     2014     1.133333
dtype: float64
Month  Year
8      2014    0.5
9      2014    1.9
10     2014    0.5
dtype: float64


In [13]:
### For liberia ###
# Keywords for New cases: New Case/s (Suspected), New Case/s (Probable), New case/s (confirmed)
# Keywords for New deaths: Newly reported deaths
# Column "Variable" gives detail of each record (row)
# Column "National" is assumed to contain the sum of data per variable
liberia_new_cases_label = ["New Case/s (Suspected)","New Case/s (Probable)", "New case/s (confirmed)"]
liberia_new_deaths_label = [ "Newly reported deaths" ]

liberia_new_cases = extract_data(ebola_data, "liberia", "Variable", liberia_new_cases_label, "National")
liberia_new_deaths = extract_data(ebola_data, "liberia", "Variable", liberia_new_deaths_label, "National")

print(liberia_new_cases)
print(liberia_new_deaths)

Month  Year
6      2014       1.333333
7      2014       3.133333
8      2014      11.166667
9      2014      51.066667
10     2014      38.433333
11     2014      15.666667
12     2014    1553.566667
dtype: float64
Month  Year
6      2014     0.466667
7      2014     1.566667
8      2014     6.966667
9      2014    28.833333
10     2014    23.433333
11     2014     6.833333
12     2014     0.000000
dtype: float64


In [14]:
### For SL ###
# Keywords for New cases: new_suspected, new_probable, new_confirmed
# Keywords for new deaths : etc_new_deaths
# Column "variable" gives detail of each record (row)
# Column "National" is assumed to contain the sum of data per variable
sl_new_cases_label = ["new_suspected", "new_probable", "new_confirmed"]
sl_new_deaths_label = [ "etc_new_deaths" ]

sl_new_cases = extract_data(ebola_data, "sl", "variable", sl_new_cases_label, "National")
sl_new_deaths = extract_data(ebola_data, "sl", "variable", sl_new_deaths_label, "National")

print(sl_new_cases)
print(sl_new_deaths)

Month  Year
8      2014    17.766667
9      2014    41.400000
10     2014    67.133333
11     2014    55.000000
12     2014     6.833333
dtype: float64
Month  Year
8      2014    3.700000
9      2014    4.200000
10     2014    3.300000
11     2014    0.400000
12     2014    0.366667
dtype: float64


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [42]:
import os

frame = pd.DataFrame()
files = os.listdir(DATA_FOLDER+"/microbiome")
i=1
for f in files:
    if(f.startswith('MID')):
        frame = frame.append(pd.read_excel("data/microbiome/"+f,names=['NAME','Value'+repr(i)], header=None))
        i+=1
        
frame = frame.groupby('NAME').sum()
#frame = frame.reset_index()  #If not used, names are used as index
frame = frame.fillna('Unknown')

#Metadata import
metadata = pd.DataFrame(pd.read_excel("data/microbiome/metadata.xls"))
legend = (metadata['SAMPLE'].fillna('Unknown')+', '+metadata['GROUP']).tolist()
frame.columns = legend

#Merges sample types
a = frame.columns.str.split(', ', expand=True).values
frame.columns = pd.MultiIndex.from_tuples([('', x[0]) if pd.isnull(x[1]) else x for x in a])
frame.columns.names = reversed(metadata.columns[1::])
frame

SAMPLE                                                        Unknown  \
GROUP                                              EXTRACTION CONTROL   
NAME                                                                    
Archaea "Crenarchaeota" Thermoprotei Acidilobal...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Acidilobal...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                  7   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc...                  2   
Archaea "Crenarchaeota" Thermoprotei Sulfolobal...                  3   
Archaea "Crenarchaeota" Thermoprotei Sulfolobal...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...                  3   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...            Unknown   
Archaea "Crenarchaeota" Thermoprotei Thermoprot...            Unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                  7   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            Unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            Unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            Unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            Unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                  1   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...            Unknown   
Archaea "Euryarchaeota" "Methanomicrobia" Metha...                  1   
Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...                  1   
Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...                  1   
Archaea "Euryarchaeota" Halobacteria Halobacter...            Unknown   
Archaea "Euryarchaeota" Halobacteria Halobacter...            Unknown   
Archaea "Euryarchaeota" Halobacteria Halobacter...            Unknown   
Archaea "Euryarchaeota" Halobacteria Halobacter...                  1   
Archaea "Euryarchaeota" Halobacteria Halobacter...                  4   
...                                                               ...   
Bacteria "Proteobacteria" Gammaproteobacteria X...            Unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            Unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            Unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            Unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            Unknown   
Bacteria "Proteobacteria" Gammaproteobacteria X...            Unknown   
Bacteria "Spirochaetes" Spirochaetes Spirochaet...                  1   
Bacteria "Synergistetes" Synergistia Synergista...                  2   
Bacteria "Synergistetes" Synergistia Synergista...                  1   
Bacteria "Synergistetes" Synergistia Synergista...                  1   
Bacteria "Tenericutes" Mollicutes Anaeroplasmat...            Unknown   
Bacteria "Tenericutes" Mollicutes Haloplasmatal...                  1   
Bacteria "Thermodesulfobacteria" Thermodesulfob...                  1   
Bacteria "Thermodesulfobacteria" Thermodesulfob...            Unknown   
Bacteria "Thermotogae" Thermotogae Thermotogale...                  7   
Bacteria "Thermotogae" Thermotogae Thermotogale...                  9   
Bacteria "Verrucomicrobia" Opitutae Opitutales ...                  1   
Bacteria "Verrucomicrobia" Opitutae Opitutales ...            Unknown   
Bacteria "Verrucomicrobia" Opitutae Puniceicocc...            Unknown   
Bacteria "Verrucomicrobia" Opitutae Puniceicocc...            Unknown   
Bacteria "Verrucomicrobia" Opitutae Puniceicocc...        

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here